## Section 0: Import Packages

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os
import PIL
import PIL.Image as Imageee
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont
%matplotlib inline

## Section 1: Camera Calibration

In [ ]:
# Process: read in images and create imagepoints and objectpoints using which we can compute distortion coefficients and
# tranformation matrix with the help of calibrateCamera OpenCV function and undistort image using OpenCV undistort function
# calib_imgs = []
objpts = []
imgpts = []
# img_cor = []
objp = np.zeros((9*6, 3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

for file in glob.glob('camera_cal/calibration*.jpg'):
#     calib_imgs.append(cv2.imread(file))
    img = cv2.imread(file)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    if ret == True:
        imgpts.append(corners)
        objpts.append(objp)        
#         img_cor.append(cv2.drawChessboardCorners(img, (9, 6), corners, ret))
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, gray.shape[::-1], None, None)

## Section 2: Function Defnitions

In [ ]:
# Camera Calibrator
def calibrator(img):
    dist = np.array([[-0.24688507,-0.02373155,-0.00109831,0.00035107,-0.00259868]])
    mtx = np.array([[1.15777818e+03, 0.00000000e+00, 6.67113857e+02],[0.00000000e+00, 1.15282217e+03, 3.86124583e+02],[0.00000000e+00,0.00000000e+00,1.00000000e+00]])
    return cv2.undistort(img, mtx, dist, None, mtx)

#==============================================================================================================================
# Color and Gradient Threshold Function
def sthresh(im):
    xthresh = [30,80] # fine
    ythresh = [0,100] # not useful
    mthresh = [20,100] # not useful
    dthresh = [0.3,1.3] # fine
    rcthresh = [220,255] # fine
    scthresh = [110,255] # fine
    
    r_img = im[:,:,2]
    s_img = (cv2.cvtColor(im,cv2.COLOR_RGB2HLS))[:,:,2]
    
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = 19)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = 19)
    abs_sobelx = (sobelx**2)**(0.5)
    abs_sobely = (sobely**2)**(0.5)
    dir_sobel = np.arctan2(abs_sobely,abs_sobelx)
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    
    sxbin = np.zeros_like(scaled_sobel)
    sybin = np.zeros_like(scaled_sobel)
    dsbin = np.zeros_like(scaled_sobel)
    msbin = np.zeros_like(scaled_sobel)
    combin1 = np.zeros_like(scaled_sobel)
    combin2 = np.zeros_like(scaled_sobel)
    combin3 = np.zeros_like(scaled_sobel)
    combin4 = np.zeros_like(scaled_sobel)
    rbin = np.zeros_like(r_img)
    sbin = np.zeros_like(s_img)    
    
    sxbin[(scaled_sobelx > xthresh[0]) & (scaled_sobelx < xthresh[1])] = 1 # peforms poorly only in lighter(whiter) roads
    sybin[(scaled_sobely > ythresh[0]) & (scaled_sobely < ythresh[1])] = 1 # sybin is not very useful compared to sxbin
    msbin[(scaled_sobel > mthresh[0]) & (scaled_sobel < mthresh[1])] = 1 # adds junk from ysobel
    dsbin[(dir_sobel > dthresh[0]) & (dir_sobel < dthresh[1])] = 1 # lot of junk with lanes
    rbin[(r_img >= rcthresh[0]) & (r_img <= rcthresh[1])] = 1 # best except shadows
    sbin[(s_img >= scthresh[0]) & (s_img <= scthresh[1])] = 1 # decent every condition
    
    # Good combos
    combin1[(((sbin==1) & (rbin==1)) | (sxbin==1))  & (dsbin==1)] = 1
    combin2[(((sbin==1) | (rbin==1)) | (sxbin==1))] = 1
    combin3[(sbin==1) | (rbin==1)] = 1
    combin4[((sbin==1) & (rbin==1)) | ((sxbin==1) & (sybin==1))] = 1
    
    return combin3

# #============================================================================================================================
# Perspective and Inverse Perspective Transform functions


def inv_persp_trans(img):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([ [200,720], [1050,720], [678,447], [600,447]])
    dst = np.float32([ [180,720], [800,720], [800,0], [180,0]])
    M = cv2.getPerspectiveTransform(dst, src)
    warp = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warp
# alternative
def ninv_persp_trans(img):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([ [200,720], [1050,720], [720,480], [550,480]])
    dst = np.float32([ [250,720], [900,720], [900,0], [250,0]])
    M = cv2.getPerspectiveTransform(dst, src)
    warp = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warp

def persp_trans(img):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([ [200,720], [1050,720], [678,447], [600,447]])
    dst = np.float32([ [180,720], [800,720], [800,0], [180,0]])
    M = cv2.getPerspectiveTransform(src,dst)
    warp = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warp
#alternative
def npersp_trans(img):
    img_size = (img.shape[1],img.shape[0])
    src = np.float32([ [200,720], [1050,720], [720,480], [550,480]])
    dst = np.float32([ [250,720], [900,720], [900,0], [250,0]])
    M = cv2.getPerspectiveTransform(src,dst)
    warp = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warp

def region_of_interest(img):
    #defining a blank mask to start with - whole image is zeros
    mask = np.zeros_like(img) 
    
    v1 = [100,img.shape[0]]
    v2 = [img.shape[1]-50,img.shape[0]]
    v3 = [img.shape[1]-50,0]
    v4 = [100,0]
    vertices=np.array([(v1,v2,v3,v4)])
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image - simply fills the color channel
    # image is in; if its a full channel, then the original image, gray, then just gray, etc
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color) # - vertices order is important to form the desired polygon shape
    # creates the channel in expressed polygon with others as zero
       
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask) #gives masked img as bitwise func adds img elements in nonzero,i.e., in polygon 
    return masked_image

#====================================================================================================================

# Find Lane Lines initially functions
def find_lane_pixels(binary_warped):

    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    out_img = np.dstack((binary_warped*255, binary_warped*255, binary_warped*255))
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    nwindows = 4
    margin = 100
    minpix = 100

    window_height = np.int(binary_warped.shape[0]//nwindows)
    nonzero = binary_warped.nonzero() # indices at which binar_warped has nonzero elements
    nonzeroy = np.array(nonzero[0]) # indices of nonzero elements of 
    # rows of binary_warped 
    nonzerox = np.array(nonzero[1]) # indices of nonzero elements of colums
    # of binary_warped
    leftx_current = leftx_base
    rightx_current = rightx_base

    #Empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    left_flag = 1
    right_flag = 1

    # Step through the left windows one by one
    for window in range(nwindows):


        win_y_low = binary_warped.shape[0] - (window+1)*window_height 
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - (margin) 
        win_xleft_high = leftx_current + (margin)  

        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 7) 

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)

        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        # Flag reset for high curvature lane ie lane out of frame    
        if len(good_left_inds) == 0:
            left_flag = 0

    for window in range(nwindows):

        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xright_low = rightx_current - (margin) 
        win_xright_high = rightx_current + (margin) 

        cv2.rectangle(out_img,(win_xright_low,win_y_low), (win_xright_high,win_y_high),(0,255,0), 7) 

        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        right_lane_inds.append(good_right_inds)

        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        # Flag reset for large curvature lane ie lane out of frame
        if len(good_right_inds) == 0:
            right_flag = 0   

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        pass

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def fit_polynomial(binary_warped):
    fit_flag = 0
    # Find our lane pixels first
    leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)

    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        fit_flag = 0
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        fit_flag = 1
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty


    return left_fit, right_fit, ploty, fit_flag
#============================================================================================================================

# After first frame lane line detection
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    fit_flag = 0
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
        
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2] 
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] 
        fit_flag = 0
    except TypeError:
    # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        fit_flag = 1
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    
    return left_fit, right_fit, ploty, fit_flag

def targeted_lane_search(img, left_fit, right_fit):

# #     For testing
#     left_fit = np.array([ -1.82904696e-05, 7.34004683e-02, 1.53926681e+02])
#     right_fit = np.array([  1.39554306e-04, -1.62273165e-01, 1.15337075e+03])
    margin = 100
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox >= left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy \
                    + left_fit[2] - margin) & (nonzerox <= left_fit[0]*nonzeroy**2 \
                    + left_fit[1]*nonzeroy + left_fit[2] + margin))
    
    right_lane_inds = ((nonzerox >= right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy \
                    + right_fit[2] - margin) & (nonzerox <= right_fit[0]*nonzeroy**2 \
                    + right_fit[1]*nonzeroy + right_fit[2] + margin))

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    if ((len(leftx) == 0) | (len(rightx) == 0)):
        print('Lines Not Detected')
        fit_flag = 1
        left_fit = []
        right_fit = []
        ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
        return left_fit, right_fit, ploty, fit_flag
    else:
        left_fit, right_fit, ploty, fit_flag = fit_poly(img.shape, leftx, lefty, rightx, righty)
    
    return left_fit, right_fit, ploty, fit_flag
#============================================================================================================================
# Curvature and vehicle lane position Function

def curvature(lfit, rfit, ploty):

    xm = 3.7/600 # meters per pixel in x
    ym = 30/720 # meters per pixel in y

    y_eval_m = np.max(ploty)*ym

    #converting fit coefficients from pixels to meters
    left_fit_cr = [lfit[0]*((xm)/(ym**2)), lfit[1]*((xm)/ym), lfit[2]]
    right_fit_cr = [rfit[0]*((xm)/(ym**2)), rfit[1]*((xm)/ym), rfit[2]]

    left_curverad = (( 1+ ((( 2*left_fit_cr[0]*y_eval_m) + left_fit_cr[1])**2))**(1.5)) / np.absolute(2*left_fit_cr[0])  ## Implement the calculation of the left line here
    right_curverad = (( 1+ ((( 2*right_fit_cr[0]*y_eval_m) + right_fit_cr[1])**2))**(1.5)) / np.absolute(2*right_fit_cr[0])  ## Implement the calculation of the right line here
    
    veh_xpos = (1280/2)*xm
    ylane_high = 720
    xleft = (lfit[0]*(ylane_high**2) + lfit[1]*(ylane_high) + lfit[2])*xm
    xright = (rfit[0]*(ylane_high**2) + rfit[1]*(ylane_high) + rfit[2])*xm
    lane_mid = xleft + ((xright - xleft)/2)
    veh_ln_pos = veh_xpos - lane_mid # in meters
    
    return left_curverad, right_curverad, veh_ln_pos, xleft, xright
#==============================================================================================================================

## Section3: LANE DETECTION


In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Define a class to receive the characteristics of each line detection and to check their sanity

history = 5 # to smoothen out the lines

class left_lane():
    # was the line detected in the last iteration
    detected = False  
    #polynomial coefficients for the most recent fit
    current_fit_coeff = []
    #radius of curvature of the line in some units
    radius_of_curvature = None 
    #x values for detected line pixels
    allx = None
    # lane start x location in meters
    xcoord = None
    # best fit coefficients by averaging best coefficients
    best_fit_coeff = []
    
    def reset():
        # was the line detected in the last iteration
        left_lane.detected = False  
        #polynomial coefficients for the most recent fit
        left_lane.current_fit_coeff = []
        #radius of curvature of the line in some units
        left_lane.radius_of_curvature = None 
        #x values for detected line pixels
        left_lane.allx = None
        # lane start x location in meters
        left_lane.xcoord = None
        # best fit coefficients by averaging best coefficients
        left_lane.best_fit_coeff = []
        
class right_lane():
    # was the line detected in the last iteration
    detected = False  
    #polynomial coefficients for the most recent fit
    current_fit_coeff = []
    #radius of curvature of the line in some units
    radius_of_curvature = None 
    #x values for detected line pixels
    allx = None
    # lane start x location in meters
    xcoord = None
    # best fit coefficients by averaging best coefficients
    best_fit_coeff = []
    
    
    def reset():
        # was the line detected in the last iteration
        right_lane.detected = False  
        #polynomial coefficients for the most recent fit
        right_lane.current_fit_coeff = []
        #radius of curvature of the line in some units
        right_lane.radius_of_curvature = None 
        #x values for detected line pixels
        right_lane.allx = None
        # lane start x location in meters
        right_lane.xcoord = None
        # best fit coefficients by averaging best coefficients
        right_lane.best_fit_coeff = []

class Sanity():
    sanity_flag = None # 0 is good, 1 is bad
    def reset():
        Sanity.sanity_flag = None
        
def lane_detector(img):     
    
    # step1: apply camera calibration to current frame
    calibrated_img = calibrator(img)    
    
    # step2: apply thresholds and convert image to binary
    thresh_bin_img = sthresh(calibrated_img)  
    
    # step3: perspective transform and masking unwanted borders
    persp_bin_img = persp_trans(thresh_bin_img)

    print('Processing New Frame....')
    print('Sanity flag is =', Sanity.sanity_flag)
    print()    
    
    # Option A:
    # if left and right lines == (empty) OR (not empty & unacceptable based on sanity checks) - initial lane line detection
    # will return y coords and left and right fit coefficents
    if (((left_lane.detected == False) & (right_lane.detected == False)) | (Sanity.sanity_flag == 1)):
        print('Sliding window being executed...')
        left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty, fit_flag = fit_polynomial(persp_bin_img)

        #step6: lane curvature and vehicle position within the lane calculation
        left_lane.radius_of_curvature, right_lane.radius_of_curvature, veh_ln_pos,left_lane.xcoord,right_lane.xcoord  = curvature(left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty)           
                       
        # Storing sane fit coeffecients in best fit objects of each lane line class
        if ((len(left_lane.best_fit_coeff) == 0) & (len(right_lane.best_fit_coeff) == 0)):
            left_lane.best_fit_coeff.append(left_lane.current_fit_coeff)
            right_lane.best_fit_coeff.append(right_lane.current_fit_coeff)
        elif ( ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) <= 0.01) & \
             ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) >= -0.01) & \
             ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) <= 1) & \
             ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) >= -1) & \
             (left_lane.radius_of_curvature <= 1200) & (right_lane.radius_of_curvature <= 1200) & \
             ((right_lane.xcoord - left_lane.xcoord) <= 4.5) & ((right_lane.xcoord - left_lane.xcoord) >= 3.5) ):
            left_lane.best_fit_coeff.append(left_lane.current_fit_coeff)
            right_lane.best_fit_coeff.append(right_lane.current_fit_coeff)
        
        # Taking average of best fits over history as current fit coefficients
        if len(left_lane.best_fit_coeff) < history:
            left_lane.current_fit_coeff = np.mean(np.array(left_lane.best_fit_coeff),axis=0)
            right_lane.current_fit_coeff = np.mean(np.array(right_lane.best_fit_coeff),axis=0)
        else:
            left_lane.current_fit_coeff = np.mean(((np.array(left_lane.best_fit_coeff))[len(left_lane.best_fit_coeff)-history:len(left_lane.best_fit_coeff)]),axis=0)
            right_lane.current_fit_coeff = np.mean(((np.array(right_lane.best_fit_coeff))[len(right_lane.best_fit_coeff)-history:len(right_lane.best_fit_coeff)]),axis=0)
       
        #Updating lane curvatures, vehicle position
        left_lane.radius_of_curvature, right_lane.radius_of_curvature, veh_ln_pos,left_lane.xcoord,right_lane.xcoord  = curvature(left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty)           

        #Sanity check        
        if ( ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) <= 0.01) & \
             ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) >= -0.01) & \
             ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) <= 1) & \
             ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) >= -1) & \
             (left_lane.radius_of_curvature <= 1200) & (right_lane.radius_of_curvature <= 1200) & \
             ((right_lane.xcoord - left_lane.xcoord) <= 4.5) & ((right_lane.xcoord - left_lane.xcoord) >= 3.5) ):
            Sanity.sanity_flag = 0
        else:
            Sanity.sanity_flag = 1
            
        right_lane.detected = True
        left_lane.detected = True
            
       
    # Option B: 
    # if left and right lines == (not empty) & (accepetable based on sanity checks) - Targeted lane detection
    # use returned y coords and left and right fit coefficents for a targeted search 
    elif ((((left_lane.detected == True) & (right_lane.detected == True)) & (Sanity.sanity_flag == 0))):
        left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty, fit_flag = targeted_lane_search(persp_bin_img, left_lane.current_fit_coeff, right_lane.current_fit_coeff)
        
        # Checking if targeted detection detected lanes
        if fit_flag == 1: # detection failed - using previous values only and raising sanity flag to perform window search
            Sanity.sanity_flag = 1
            if len(left_lane.best_fit_coeff) < history:
                left_lane.current_fit_coeff = np.mean(np.array(left_lane.best_fit_coeff),axis=0)
                right_lane.current_fit_coeff = np.mean(np.array(right_lane.best_fit_coeff),axis=0)
            else:
                left_lane.current_fit_coeff = np.mean(((np.array(left_lane.best_fit_coeff))[len(left_lane.best_fit_coeff)-history:len(left_lane.best_fit_coeff)]),axis=0)
                right_lane.current_fit_coeff = np.mean(((np.array(right_lane.best_fit_coeff))[len(right_lane.best_fit_coeff)-history:len(right_lane.best_fit_coeff)]),axis=0)
       
            left_lane.radius_of_curvature, right_lane.radius_of_curvature, veh_ln_pos,left_lane.xcoord,right_lane.xcoord  = curvature(left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty)               
            right_lane.detected = False
            left_lane.detected = False
        
        elif fit_flag == 0:           
            
            print('Targeted Search being executed...')
        
            #step6: lane curvature and vehicle position within the lane calculation
            left_lane.radius_of_curvature, right_lane.radius_of_curvature, veh_ln_pos,left_lane.xcoord,right_lane.xcoord  = curvature(left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty)               
            
            # Storing sane fit coeffecients in best fit objects of each lane line class
            if ( ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) <= 0.01) & \
                 ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) >= -0.01) & \
                 ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) <= 1) & \
                 ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) >= -1) & \
                 (left_lane.radius_of_curvature <= 1200) & (right_lane.radius_of_curvature <= 1200) & \
                 ((right_lane.xcoord - left_lane.xcoord) <= 4.5) & ((right_lane.xcoord - left_lane.xcoord) >= 3.5) ):
                left_lane.best_fit_coeff.append(left_lane.current_fit_coeff)
                right_lane.best_fit_coeff.append(right_lane.current_fit_coeff)
            
            # Taking average of best fits over history as current fit coefficients
            if len(left_lane.best_fit_coeff) < history:
                left_lane.current_fit_coeff = np.mean(np.array(left_lane.best_fit_coeff),axis=0)
                right_lane.current_fit_coeff = np.mean(np.array(right_lane.best_fit_coeff),axis=0)
            else:
                left_lane.current_fit_coeff = np.mean(((np.array(left_lane.best_fit_coeff))[len(left_lane.best_fit_coeff)-history:len(left_lane.best_fit_coeff)]),axis=0)
                right_lane.current_fit_coeff = np.mean(((np.array(right_lane.best_fit_coeff))[len(right_lane.best_fit_coeff)-history:len(right_lane.best_fit_coeff)]),axis=0)
            
            # Updating curvatures, vehicle position
            left_lane.radius_of_curvature, right_lane.radius_of_curvature, veh_ln_pos,left_lane.xcoord,right_lane.xcoord  = curvature(left_lane.current_fit_coeff, right_lane.current_fit_coeff, ploty)               
    

            #Sanity check        
            if ( ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) <= 0.01) & \
                 ((left_lane.current_fit_coeff[0]-right_lane.current_fit_coeff[0]) >= -0.01) & \
                 ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) <= 1) & \
                 ((left_lane.current_fit_coeff[1]-right_lane.current_fit_coeff[1]) >= -1) & \
                 (left_lane.radius_of_curvature <= 1200) & (right_lane.radius_of_curvature <= 1200) & \
                 ((right_lane.xcoord - left_lane.xcoord) <= 4.5) & ((right_lane.xcoord - left_lane.xcoord) >= 3.5) ):
                Sanity.sanity_flag = 0
            else:
                Sanity.sanity_flag = 1
            
    #step7: Return frame with lane shaded, displaying lane curvature and vehicle postion within the lane
    left_lane.allx = left_lane.current_fit_coeff[0]*(ploty**2) + left_lane.current_fit_coeff[1]*ploty + left_lane.current_fit_coeff[2]
    right_lane.allx = right_lane.current_fit_coeff[0]*(ploty**2) + right_lane.current_fit_coeff[1]*ploty + right_lane.current_fit_coeff[2]
    left_lane.ally = ploty
    right_lane.ally = ploty
    #=====================================================================================================
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    window_img = np.zeros_like(img)
    # Generate a polygon to illustrate the lane
    # And recast the x and y points into usable format for cv2.fillPoly()    
    left_line = np.array([np.transpose(np.vstack([left_lane.allx, left_lane.ally]))])
    right_line = np.array([np.flipud(np.transpose(np.vstack([right_lane.allx, right_lane.ally])))])
    lane_line_pts = np.hstack((left_line, right_line))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([lane_line_pts]), (0, 100, 255))
    result = cv2.addWeighted(img, 1, inv_persp_trans(window_img), 0.4, 0)
    
    x, y = 1280, 720
    radius = (min(left_lane.radius_of_curvature, right_lane.radius_of_curvature)).astype(int).astype(str)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result,'Radius of Curvature: ',(10, y//8), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
    cv2.putText(result,radius,(500, y//8), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
    cv2.putText(result,'m',(630, y//8), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
    veh_ln_pos = np.round(veh_ln_pos,2)
    if veh_ln_pos >= 0:
        veh = veh_ln_pos.astype(str)
        cv2.putText(result,'Vehicle is ',(10, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
        cv2.putText(result,veh,(250, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
        cv2.putText(result,'m',(360, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
        cv2.putText(result,' right of center',(390, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
    if veh_ln_pos < 0:
        veh = (-1*veh_ln_pos).astype(str)
        cv2.putText(result,'Vehicle is ',(10, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
        cv2.putText(result,veh,(250, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
        cv2.putText(result,'m',(360, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)
        cv2.putText(result,' left of center',(390, y//5), font, 1.5, (255, 255, 255), 4, cv2.LINE_AA)

    return result

## Project Video

In [ ]:
# Resetting class attributes
left_lane.reset()
right_lane.reset()
Sanity.reset()

project_output = 'Project_Output_Videos/project_video_final7.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(lane_detector) #NOTE: this function expects color images!!
%time white_clip.write_videofile(project_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

## Challenge Video

In [ ]:
challenge_output = 'Project_Output_Videos/challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip2 = VideoFileClip("challenge_video.mp4")
white_clip = clip2.fl_image(lane_detector) #NOTE: this function expects color images!!
%time white_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

## HARDER CHALLENGE

In [ ]:
harder_challenge_output = 'Project_Output_Videos/harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip3 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip3.fl_image(lane_detector) #NOTE: this function expects color images!!
%time white_clip.write_videofile(harder_challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))

## MISC

In [ ]:
# # Creating Frame Images from Project Video 
# # Read the video from specified path 
# cam = cv2.VideoCapture("project_video.mp4") 
  
# try:       
#     # creating a folder named data 
#     if not os.path.exists('Video Frame Images'): 
#         os.makedirs('Video Frame Images') 
  
# # if not created then raise error 
# except OSError: 
#     print ('Error: Creating directory of data') 
  
# # frame 
# currentframe = 0
  
# while(True): 
      
#     # reading from frame 
#     ret,frame = cam.read() 
  
#     if ret: 
#         # if video is still left continue creating images 
#         name = './Video Frame Images/frame' + str(currentframe) + '.jpg'
#         print ('Creating...' + name) 
  
#         # writing the extracted images 
#         cv2.imwrite(name, frame) 
  
#         # increasing counter so that it will 
#         # show how many frames are created 
#         currentframe += 1
#     else: 
#         break


# # Straight lane images
# strt = []
# for image in glob.glob('test_images/straight_lines*.jpg'):
#     strt.append(cv2.imread(image))
    
# ctest = []
# for image in test:
#     ctest.append((image))

# r_img = ctest[3][:,:,2]
# s_img = (cv2.cvtColor(ctest[3],cv2.COLOR_RGB2HLS))[:,:,2]

# plt.imshow(cv2.imread('Video Frame Images/frame1050.jpg'))

# Test Images
# test = []
# for image in glob.glob('test_images/test*.jpg'):
#     test.append(cv2.imread(image))


# def find_lane_pixels(binary_warped):

#     histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
#     out_img = np.dstack((binary_warped*255, binary_warped*255, binary_warped*255))
#     midpoint = np.int(histogram.shape[0]//2)
#     leftx_base = np.argmax((histogram[:midpoint]))
#     print(leftx_base)
#     rightx_base = np.argmax((histogram[midpoint:])) + midpoint
#     print(rightx_base)

#     # HYPERPARAMETERS
#     nwindows = 4
#     margin = 100
#     minpix = 100

#     window_height = np.int(binary_warped.shape[0]//nwindows)
#     nonzero = binary_warped.nonzero() # indices at which binar_warped has nonzero elements
#     nonzeroy = np.array(nonzero[0]) # indices of nonzero elements of 
#     # rows of binary_warped 
#     nonzerox = np.array(nonzero[1]) # indices of nonzero elements of colums
#     # of binary_warped
#     leftx_current = leftx_base
#     rightx_current = rightx_base

#     #Empty lists to receive left and right lane pixel indices
#     left_lane_inds = []
#     right_lane_inds = []

#     left_flag = 1
#     right_flag = 1

#     # Step through the left windows one by one
#     for window in range(nwindows):

#         if left_flag == 0:
#             break

#         win_y_low = binary_warped.shape[0] - (window+1)*window_height 
#         win_y_high = binary_warped.shape[0] - window*window_height
#         win_xleft_low = leftx_current - (margin) 
#         win_xleft_high = leftx_current + (margin)  

# #         cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 7) 

#         good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]

#         left_lane_inds.append(good_left_inds)

#         if len(good_left_inds) > minpix:
#             leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
#         # Flag reset for high curvature lane ie lane out of frame    
#         if len(good_left_inds) == 0:
#             left_flag = 0

#     for window in range(nwindows):
#         if right_flag == 0:
#             break

#         win_y_low = binary_warped.shape[0] - (window+1)*window_height
#         win_y_high = binary_warped.shape[0] - window*window_height
#         win_xright_low = rightx_current - (margin) 
#         win_xright_high = rightx_current + (margin) 

# #         cv2.rectangle(out_img,(win_xright_low,win_y_low), (win_xright_high,win_y_high),(0,255,0), 7) 

#         good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

#         right_lane_inds.append(good_right_inds)

#         if len(good_right_inds) > minpix:
#             rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
#         # Flag reset for large curvature lane ie lane out of frame
#         if len(good_right_inds) == 0:
#             right_flag = 0   

#     # Concatenate the arrays of indices (previously was a list of lists of pixels)
#     try:
#         left_lane_inds = np.concatenate(left_lane_inds)
#         right_lane_inds = np.concatenate(right_lane_inds)
#     except ValueError:
#         pass

#     leftx = nonzerox[left_lane_inds]
#     lefty = nonzeroy[left_lane_inds] 
#     rightx = nonzerox[right_lane_inds]
#     righty = nonzeroy[right_lane_inds]

#     return leftx, lefty, rightx, righty, out_img

# def fit_polynomial(binary_warped):
#     # Find our lane pixels first
#     leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

#     ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
#     left_fit = np.polyfit(lefty,leftx,2)
#     right_fit = np.polyfit(righty,rightx,2)

#     # Generate x and y values for plotting
#     ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
#     try:
#         left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
#         right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
#         fit_flag = 0
#     except TypeError:
#         # Avoids an error if `left` and `right_fit` are still none or incorrect
#         print('The function failed to fit a line!')
#         fit_flag = 1
#         left_fitx = 1*ploty**2 + 1*ploty
#         right_fitx = 1*ploty**2 + 1*ploty

# #         ## Visualization ##
# #     # Create an image to draw on and an image to show the selection window
# #     window_img = np.zeros_like(out_img)
# #     # Color in left and right line pixels
# #     out_img[lefty, leftx] = [255, 0, 0]
# #     out_img[righty, rightx] = [0, 0, 255]
    
# #     # Generate a polygon to illustrate the search window area
# #     # And recast the x and y points into usable format for cv2.fillPoly()    
# #     left_line = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
# #     right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
# #                               ploty])))])
# #     lane_line_pts = np.hstack((left_line, right_line))
    
# #     # Draw the lane onto the warped blank image
# #     cv2.fillPoly(window_img, np.int_([lane_line_pts]), (0,255, 0))
# #     result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

#     return left_fitx, right_fitx, ploty, leftx, lefty, rightx, righty

# left_fitx, right_fitx, ploty, leftx, lefty, rightx, righty = fit_polynomial(npersp_trans(sthresh(calibrator(cv2.imread('Video Frame Images/frame577.jpg')))))

#     ## Visualization ##
# # Create an image to draw on and an image to show the selection window
# window_img = np.zeros_like(ctest[3])
# # Generate a polygon to illustrate the lane
# # And recast the x and y points into usable format for cv2.fillPoly()    
# left_line = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
# right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
#                           ploty])))])
# lane_line_pts = np.hstack((left_line, right_line))

# # Draw the lane onto the warped blank image
# cv2.fillPoly(window_img, np.int_([lane_line_pts]), (0,255, 0))
# # Color lane line pixels
# # window_img[lefty, leftx] = [255, 0, 0]
# # window_img[righty, rightx] = [0, 0, 255]


# result = cv2.addWeighted(ctest[3], 1, ninv_persp_trans(window_img), 0.3, 0)
# plt.imshow(result)

# # plt.imshow(fit_polynomial(persp_trans(sthresh(calibrator(ctest[3])))))

# #         ## Visualization ##
# #     # Create an image to draw on and an image to show the selection window
# #     out_img = np.dstack((img, img, img))*255
# #     window_img = np.zeros_like(out_img)
# #     # Color in left and right line pixels
# #     out_img[lefty, leftx] = [255, 0, 0]
# #     out_img[righty, rightx] = [0, 0, 255]
    
# #     # Generate a polygon to illustrate the search window area
# #     # And recast the x and y points into usable format for cv2.fillPoly()    
# #     left_line = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
# #     right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
# #                               ploty])))])
# #     lane_line_pts = np.hstack((left_line, right_line))
    
# #     # Draw the lane onto the warped blank image
# #     cv2.fillPoly(window_img, np.int_([lane_line_pts]), (0,255, 0))
# #     result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

# Some fit coeffs
# frame 1: 
# Left Fit Coeffs are  [ -3.71064484e-04   5.86213470e-01   1.42034320e+01]
# Right Fit Coeffs are  [ -1.12240616e-04   3.92790208e-01   6.96780876e+02]

# frame 250:
# Left Fit Coeffs are  [ -3.89719575e-04   6.27087852e-01  -5.72137681e+01]
# Right Fit Coeffs are  [ -3.74064225e-04   7.26779421e-01   5.31256824e+02]

# frame 500:
# Left Fit Coeffs are  [ -2.12054828e-05   4.11008919e-02   1.70445694e+02]
# Right Fit Coeffs are  [  2.27968498e-04  -1.79623561e-01   9.03068022e+02]

# frame 400:
# Left Fit Coeffs are  [  1.03197177e-05   4.17584769e-02   1.85013763e+02]
# Right Fit Coeffs are  [  5.59604526e-05   3.98701154e-03   8.55023545e+02]
        